## Homework 3 - Which book would you recommend?

-------------

### Importing related libraries

In [7]:
from bs4 import BeautifulSoup
import requests
import lxml
import csv
import re
import os
import pandas as pd
from langdetect import detect
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
import pandas as pd
import glob
import json
import functions

## 1. Data collection

#### 1.1. Get the list of books

* From [best books ever list](https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1) we want to collect the url associated to each book in the list and retrieve only the urls of the books listed in the first 300 pages.



* The output of this step is a `.txt` file whose single line corresponds to a book's url.

In [ ]:
#inizialize an empty list
url = []

#for each page, save the corresponding web page, find the anchor elements and save the corresponding tags  
for i in range(1, 301):
    page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+ str(i))
    soup = BeautifulSoup(page.content, features='lxml')
    tag_a = soup.find_all('a', {"class": "bookTitle"}, itemprop = "url")
    
#for each book, save the corresponding url into the array
    for j in range(0,100):
        url.append("https://www.goodreads.com"+ tag_a[j]['href'])
        
#create a txt file where for each row there is a book's url 
with open("url.txt", 'w') as f:
    f.write("\n".join(map(str, url)))


#### 1.2. Crawl books

1. Download the html corresponding to each of the collected urls.


2. After you collect a single page, immediatly save its `html` in a file. In this way, if your program stops, for any reason, you will not loose the data collected up to the stopping point.


3. Organize the entire set of downloaded `html` pages into folders. Each folder will contain the `htmls` of the books in page 1, page 2, ... of the list of books.

# Warning :

### Do not run the below cell because it'll download over 20 GB files !
#### Before running modify the `for` loop range to a lower number (e.g. 3) 

In [ ]:
#First we open the url.txt file, reading the lines and then close the file
f = open("url.txt", "r")
lines = f.readlines()
f.close()

#Setting our parent directory and the directory associated with each page number from the list
## PLEASE CHANGE THE PARENT DIRECTORY ACCORDING TO YOUR SYSTEM
parent_dir = "C:/Users/engme/OneDrive/Desktop/ALL Materials/Data Science - Sapienza/1st Semester/ALGORITHMIC METHODS OF DATA MINING AND LABORATORY/Labs/6/Links"
directory = "Page_"
page_num = 0


#Looping for 300 times that corresponds to the number of pages
for i in range(300):
    #Incrementing page number according to each page
    page_num += 1
    #Setting the current working directory
    directory = "Page_" + str(page_num)
    #Setting the main path to create the directory
    path = os.path.join(parent_dir, directory)
    #Creating new directory
    os.makedirs(path)
    
    #Looping for 100 times, which is the number of articles per page
    for i in range(100):
        #Selecting the corresponding link
        link = lines[i][:-1]
        #Dowloading the article
        r = requests.get(link, allow_redirects=True)
        #Setting the name file to keep the track
        file_name = "article_" + str(i+1) + ".html"
        #Saving the html file to its corresponding directory
        open(parent_dir + "/" + directory + "/" + file_name, 'wb').write(r.content)

-----------

#### 1.3 Parse downloaded pages

* Extracting the books information for each book as following:

1. Title (to save as `bookTitle`)
2. Series (to save as `bookSeries`)
3. Author(s), the first box in the picture below (to save as `bookAuthors`)
4. Ratings, average stars (to save as `ratingValue`)
5. Number of givent ratings (to save as `ratingCount`)
6. Number of reviews (to save as `reviewCount`)
7. The entire plot (to save as `Plot`)
8. Number of pages (to save as `NumberofPages`)
9. Published (Publishing Date)
10. Characters
11. Setting
12. Url

In [ ]:
## Custom function designed to 
parent_dir = "C:/Users/engme/OneDrive/Desktop/ALL Materials/Data Science - Sapienza/1st Semester/ALGORITHMIC METHODS OF DATA MINING AND LABORATORY/Labs/6"

functions.info_parser(parent_dir, pages=300, tsv_articles= "tsv_articles", links= "Links", url= 'url')

All tsv files generated sucessfully in tsv_articles directory

--------------

## 2. Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the books that match the query.

First, you must pre-process all the information collected for each book by :



1. Removing stopwords
2. Removing punctuation
3. Stemming
4. Anything else you think it's needed


For this purpose, you can use the [nltk](https://www.nltk.org/) library.

In [3]:
#Creating a .csv files out of all .tsv files

#parent_dir = "C:/Users/engme/OneDrive/Desktop/ALL Materials/Data Science - Sapienza/1st Semester/ALGORITHMIC METHODS OF DATA MINING AND LABORATORY/Labs/6"
#tsv_folder = "tsv_articles"




# The function to create .csv file

####df_tsv = functions.create_csv(parent_dir, tsv_folder, export_csv= False)


#Reading the created .csv file
df_tsv = pd.read_csv("combined_csv.csv", usecols=["bookTitle", "Plot", "Url"])

In [8]:
# Creating the inverted index and vocabulary dictionaries

inv_index1, vocabulary = functions.create_dictionary_plot(df_tsv, export_json = False)

In [13]:
query = input("Please Enter your query: ")
functions.Search_Engine1(query, df_tsv, vocabulary, inv_index1, results= 20)

Please Enter your query: harry potter azkaban


,bookTitle,Plot,Url
6198,The Harry Potter Collection 1-4,"The exciting tales of Harry Potter, the young ...",https://www.goodreads.com/book/show/968.The_Da...
23166,The Harry Potter trilogy,This box set collects hard cover editions Har...,https://www.goodreads.com/book/show/1953.A_Tal...
25775,Harry Potter and the Prisoner of Azkaban,"For twelve long years, the dread fortress of A...",https://www.goodreads.com/book/show/5.Harry_Po...
23590,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",Box Set containing Harry Potter and the Sorcer...,https://www.goodreads.com/book/show/3636.The_G...
